In [114]:
import pickle
from collections import namedtuple
from typing import List
import numpy as np
import random
from tqdm import tqdm_notebook as tqdm

Quint = namedtuple('Quint', 's p o qp qe')

In [82]:
# Load data from disk
with open('./parsed_raw_data.pkl', 'rb') as f:
    raw_data = pickle.load(f)
    
entities, predicates = [], []

for quint in raw_data:
    entities += [quint[0], quint[2]]
    if quint[4]:
        entities.append(quint[4])
        
    predicates.append(quint[1])
    if quint[3]: 
        predicates.append(quint[3])
    
entities = list(set(entities))
predicates = list(set(predicates))
        
print(len(entities), len(predicates))

174951 659


In [116]:
def sample_negatives(quint: Quint, probs: List[float]) -> Quint:
    """ probs: [ p(s), p(r), p(o), p(q) ] """
    assert np.sum(probs)==1.0
    assert len(probs) == 4
    # print(probs)
    l = np.random.choice(["s", "p", "o", "q"], 1, p=probs)
    if l[0] == "s":
        while True:
            new_s = random.choice(entities)
            q = Quint(s=new_s, p=quint[1], o=quint[2], qp=quint[3], qe=quint[4])
            if q not in raw_data:
                return q
    elif l[0] == "p":
        while True:
            new_p = random.choice(predicates)
            q = Quint(s=quint[0], p=new_p, o=quint[2], qp=quint[3], qe=quint[4])
            if q not in raw_data:
                return q
    elif l[0] == "o":
        while True:
            new_o = random.choice(entities)
            q = Quint(s=quint[0], p=quint[1], o=new_o, qp=quint[3], qe=quint[4])
            if q not in raw_data:
                return q
    elif l[0] == "q":
        if quint[3]:
            if np.random.random() > 0.5:
                # sample qp
                while True:
                    qp = random.choice(predicates)
                    q = Quint(s=quint[0], p=quint[1], o=quint[2], qp=qp, qe=quint[4])
                    if q not in raw_data:
                        return q
            else:
                while True:
                    qe = random.choice(entities)
                    q = Quint(s=quint[0], p=quint[1], o=quint[2], qp=quint[3], qe=qe)
                    if q not in raw_data:
                        return q
        else:
            while True:
                qp = random.choice(predicates)
                qe = random.choice(entities)
                q = Quint(s=quint[0], p=quint[1], o=quint[2], qp=qp, qe=qe)
                if q not in raw_data:
                    return q
  

In [118]:
probs = [0.3, 0.0, 0.3, 0.4]
q_neg = sample_negatives(raw_data[0], probs)
print(q_neg)

Quint(s='Q636', p='P27', o='Q145', qp='P4185', qe='Q18557436')


In [108]:
l = np.random.choice(["s", "p", "o", "q"], 1000, p=probs)
print(l[0])
unique, counts = np.unique(l, return_counts=True)
dict(zip(unique, counts))
#l.count("s"), l.count("p"), l.count("o"), l.count("q")

s


{'o': 311, 'q': 390, 's': 299}

In [119]:
negative_samples = []
for q in tqdm(raw_data):
    negative_samples.append(sample_negatives(q, probs))

KeyboardInterrupt: 